In [ ]:
import numpy as np

# Arrival time of poisson process with lambda
def arrival_time(rate):
    return np.random.exponential(1/rate)

# Queueing 
class Queue:
    
    def __init__(self):
        self.level = [0, 0, 0]
        pass
    
    def add_patient(self, emergency_level):
        self.level[emergency_level] += 1
    
class Staff:
    def __init__(self, staff):
        self.staff = staff
        pass
        

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# ------------------------------
# Parameters
# ------------------------------
L = 200           # road length
dt = 0.1
lambda_rate = {'N':0.12, 'S':0.12, 'E':0.08, 'W':0.08}

# ------------------------------
# Car and TrafficLight Classes
# ------------------------------
class Car:
    def __init__(self, direction):
        self.direction = direction
        self.position = L
        self.speed = 20
        self.waiting_time = 0
        self.waiting = False

    def update(self, dt, is_green):
        # Close to stop line → must stop
        stop_zone = 30
        
        if not is_green and self.position < stop_zone:
            self.waiting = True
            self.waiting_time += dt
        else:
            self.waiting = False
            self.position -= self.speed * dt

class TrafficLight:
    def __init__(self, green_NS=30, green_EW=30):
        self.green_NS = green_NS
        self.green_EW = green_EW
        self.cycle = green_NS + green_EW

    def is_green(self, t, direction):
        phase = t % self.cycle
        if phase < self.green_NS:
            return direction in ('N','S')
        return direction in ('E','W')


# ------------------------------
# Simulation State
# ------------------------------
lanes = {d: [] for d in ['N', 'S', 'E', 'W']}
next_arrival = {d: np.random.exponential(1/lambda_rate[d]) for d in lanes}
light = TrafficLight()

t = 0

# ------------------------------
# Plot Setup
# ------------------------------
fig, ax = plt.subplots(figsize=(5, 5))
ax.set_xlim(-L, L)
ax.set_ylim(-L, L)
ax.set_aspect('equal')

text_stats = ax.text(-150, 150, "", fontsize=8)

car_plots = {d: [] for d in lanes}

# ------------------------------
# Animation Update Function
# ------------------------------
def update(frame):
    global t, next_arrival

    t += dt

    # Spawn new cars
    for d in lanes:
        next_arrival[d] -= dt
        if next_arrival[d] <= 0:
            lanes[d].append(Car(d))
            next_arrival[d] = np.random.exponential(1/lambda_rate[d])

    # Update cars
    for d, lane in lanes.items():
        green = light.is_green(t, d)
        for car in lane:
            car.update(dt, green)

        # Remove cars that passed intersection
        lanes[d] = [c for c in lane if c.position > -50]

    # Redraw cars
    ax.collections.clear()

    for d, lane in lanes.items():
        x, y = [], []
        colors = []

        for car in lane:
            if d == 'N': x.append(0);   y.append(car.position)
            if d == 'S': x.append(0);   y.append(-car.position)
            if d == 'E': x.append(car.position); y.append(0)
            if d == 'W': x.append(-car.position); y.append(0)
            colors.append("red" if car.waiting else "green")

        ax.scatter(x, y, c=colors, s=40)

    # Stats
    all_cars = [car for lane in lanes.values() for car in lane]
    num_cars = len(all_cars)
    num_waiting = sum(car.waiting for car in all_cars)
    avg_wait = np.mean([car.waiting_time for car in all_cars]) if num_cars else 0

    text_stats.set_text(
        f"t={t:.1f}s\n"
        f"# of cars={num_cars}\n"
        f"# of waiting cars={num_waiting}\n"
        f"avg waiting={avg_wait:.2f}s"
    )

    return []

ani = FuncAnimation(fig, update, interval=50)
plt.show()
